In [1]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from geopy.distance import distance
import math

In [2]:
cluster = Cluster()
session = cluster.connect('challenge')

# Seleccionar los restaurantes que se encuentren el el vecindario: Arden Heights

In [3]:
id_vencindario = '55cb9c666c522cafdb053ac0'

In [4]:
result = session.execute(
    f"select id_restaurant from restaurantsbyneighborhood where id_neighborhood = '{id_vencindario}'"
)

In [5]:
restaurants_ids_str = map(lambda row: f"'{row[0]}'", result)
restaurants_ids_str = ','.join(restaurants_ids_str)
restaurants_ids_str = '(' + restaurants_ids_str + ')'

In [6]:
result  = session.execute('select * from restaurants where id in '+ restaurants_ids_str)

In [7]:
for row in result: 
    print(row)

Row(id='55cba2476c522cafdb053f77', location=[40.55274360000001, -74.1914658], name='Villa Monte Pizzeria', type='Point')
Row(id='55cba2476c522cafdb054221', location=[40.5530453, -74.19261519999999], name='Bagel Deli / Village Green Deli', type='Point')
Row(id='55cba2476c522cafdb0561c9', location=[40.5523287, -74.1910312], name='Genki Sushi', type='Point')
Row(id='55cba2476c522cafdb057bf5', location=[40.5530453, -74.19261519999999], name='Peking Taste', type='Point')
Row(id='55cba2476c522cafdb058a17', location=[40.5530453, -74.19261519999999], name="Moe'S Southwest Griil", type='Point')
Row(id='55cba2476c522cafdb058d37', location=[40.5530453, -74.19261519999999], name='Subway', type='Point')
Row(id='55cba2476c522cafdb058f15', location=[40.5625848, -74.19097219999999], name='Dugout Pub West', type='Point')


## Encontrar restaurantes en una distancia específica respecto a un punto concreto
* Distancia = 1km
* Punto = [40.6973697290538, -73.94378455587042]

In [8]:
distancia = 1  # km
puntoInicial = [40.6973697290538, -73.94378455587042]  # Latitud, Longitud

In [9]:
result = session.execute('select * from restaurants')
filtrado = []
for row in result:
    if distance(puntoInicial, row[1]).km <= distancia:
        filtrado.append([
            row[0], row[1], row[2],
            round(distance(puntoInicial, row[1]).km, 3)
        ])

In [10]:
print('Numero de restaurantes dentro del límite', len(filtrado))
print(
    f'Imprimir restaurantes a distancia {distancia} desde {puntoInicial}:\n\n')

filtrado.sort(key=lambda item: item[3])
for ident, localizacion, nombre_restaurante, distancia in filtrado:
    print(
        f'El restaurante "{nombre_restaurante}" con coordenadas {localizacion} se encuentra a {distancia}km del punto inicial.\n'
    )

Numero de restaurantes dentro del límite 124
Imprimir restaurantes a distancia 1 desde [40.6973697290538, -73.94378455587042]:


El restaurante "Migdalia Place" con coordenadas [40.698466, -73.944343] se encuentra a 0.131km del punto inicial.

El restaurante "Elisa Cafe" con coordenadas [40.695837, -73.943679] se encuentra a 0.17km del punto inicial.

El restaurante "Crown Fried Chicken" con coordenadas [40.695896, -73.942544] se encuentra a 0.194km del punto inicial.

El restaurante "Genao" con coordenadas [40.699517, -73.944503] se encuentra a 0.246km del punto inicial.

El restaurante "Towers Cafe" con coordenadas [40.699474, -73.94279399999999] se encuentra a 0.248km del punto inicial.

El restaurante "Crown Fried Chicken" con coordenadas [40.6961798, -73.9467093] se encuentra a 0.28km del punto inicial.

El restaurante "Mike'S Pizza" con coordenadas [40.695877, -73.946589] se encuentra a 0.289km del punto inicial.

El restaurante "China City Restaurant" con coordenadas [40.695867,

In [11]:
cluster.shutdown()